In [1]:
import sys
sys.executable

'/usr/local/opt/python/bin/python3.7'

In [26]:
import numpy as np
from scipy.spatial import KDTree
import matplotlib

from collections import defaultdict
from heapq import *

In [16]:
# SampleFree Implementation

In [22]:
def SampleFree(n, d, x_init, x_goal, distr=None):
    
    # start with init point (index 0)
    V = [x_init]
    
    for i in range(n):
        
        # for now, only uniform sampling
        V.append(np.random.uniform(size=d))
        
    # add the goal point
    V.append(x_goal)
    
    return V

In [15]:
# Near Implementation

In [31]:
# This will compute X_near for all vertices in V in one go
def Near(V, r_n):
    
    # Create a KD Tree first
    KDT = KDTree(data=V)
    
    # Create the KD Tree to search against
    search_against = KDTree(data=V)

    # run query_ball_tree
    results = KDT.query_ball_tree(other=search_against, r=r_n)
    
    # construct edge set
    edges = []
    for i in range(len(V)):
        for j in results[i]:
            if i == j:
                continue
            # here, i and j are indices in V
            distance = np.linalg.norm(V[i] - V[j], ord=None)
            edges.append((i, j, distance))
            
    return edges

In [4]:
# ShortestPath Implementation

In [8]:
class Heap(): 
# Copied from G2G :^)
  
    def __init__(self): 
        self.array = [] 
        self.size = 0
        self.pos = [] 
  
    def newMinHeapNode(self, v, dist): 
        minHeapNode = [v, dist] 
        return minHeapNode 
  
    # Swap two nodes
    def swapMinHeapNode(self,a, b): 
        t = self.array[a] 
        self.array[a] = self.array[b] 
        self.array[b] = t 
  
    # Heapify at index idx
    def minHeapify(self, idx): 
        smallest = idx 
        left = 2*idx + 1
        right = 2*idx + 2
  
        if left < self.size and self.array[left][1] < self.array[smallest][1]: 
            smallest = left 
  
        if right < self.size and self.array[right][1] < self.array[smallest][1]: 
            smallest = right 
  
        # The nodes to be swapped in min heap if idx is not smallest 
        if smallest != idx: 
  
            # Swap positions 
            self.pos[ self.array[smallest][0] ] = idx 
            self.pos[ self.array[idx][0] ] = smallest 
  
            # Swap nodes 
            self.swapMinHeapNode(smallest, idx) 
            self.minHeapify(smallest) 
  
    # Popping minimum node
    def extractMin(self): 
  
        # Return NULL wif heap is empty 
        if self.isEmpty() == True: 
            return
  
        # Store the root node 
        root = self.array[0] 
  
        # Replace root node with last node 
        lastNode = self.array[self.size - 1] 
        self.array[0] = lastNode 
  
        # Update position of last node 
        self.pos[lastNode[0]] = 0
        self.pos[root[0]] = self.size - 1
  
        # Reduce heap size and heapify root 
        self.size -= 1
        self.minHeapify(0) 
  
        return root 
  
    def isEmpty(self): 
        return True if self.size == 0 else False
  
    def decreaseKey(self, v, dist): 
  
        # Get the index of v in  heap array 
        i = self.pos[v] 
  
        # Get the node and update its dist value 
        self.array[i][1] = dist 
  
        # Travel up while the complete tree is  
        # not hepified. This is a O(Logn) loop 
        while i > 0 and self.array[i][1] < self.array[(i - 1) / 2][1]: 
  
            # Swap this node with its parent 
            self.pos[ self.array[i][0] ] = (i-1)/2
            self.pos[ self.array[(i-1)/2][0] ] = i 
            self.swapMinHeapNode(i, (i - 1)/2 ) 
  
            # move to parent index 
            i = (i - 1) / 2; 
  
    # A utility function to check if a given  
    # vertex 'v' is in min heap or not 
    def isInMinHeap(self, v): 
  
        if self.pos[v] < self.size: 
            return True
        return False

In [114]:
def dijkstra_revised(edges, init, goal):
    g = defaultdict(list)
    for l,r,c in edges:
        g[l].append((c,r))

    # dist records the min value of each node in heap.
    q, seen, dist = [(0,init,())], set(), {init: 0}
    while q:
        (cost,v1,path) = heappop(q)
        if v1 in seen: continue

        seen.add(v1)
        path += (v1,)
        if v1 == goal: return (cost, path)

        for c, v2 in g.get(v1, ()):
            if v2 in seen: continue

            # Not every edge will be calculated. The edge which can improve the value of node in heap will be useful.
            if v2 not in dist or cost+c < dist[v2]:
                dist[v2] = cost+c
                heappush(q, (cost+c, v2, path))

    return float("inf")

def ShortestPath(edges, init, goal):
    output = dijkstra(edges, init, goal)
    #assert not isinstance(output, float)
    try:
        return output[0]
    except:
        if isinstance(output, float):
            print("Shortest path does not exist")
        else:
            print("An error occurred")

In [115]:
# gPRM

In [119]:
D = 2
n = 10000
r_n = 0.05

x_init = np.array([0.1] * D)
x_goal = np.array([0.9] * D)

V = SampleFree(n, D, x_init, x_goal)

In [120]:
E = Near(V, r_n)

In [121]:
length = ShortestPath(E, 0, n+1)

In [122]:
# check
print('gPRM output: ' + str(length))
print('true distance: ' + str(np.linalg.norm(x_init - x_goal, ord=None)))
print('error: +' + str(length - np.linalg.norm(x_init - x_goal, ord=None)))

gPRM output: 1.1328495294871275
true distance: 1.1313708498984762
error: +0.0014786795886512216
